In [0]:
%sql
DESCRIBE DETAIL classes.bronze.resultados_2024

In [0]:
%sql
select * from classes.bronze.resultados_2024

In [0]:
df = spark.table("classes.bronze.resultados_2024")

display(df)

In [0]:
%sql
USE CATALOG classes;
USE SCHEMA bronze;


In [0]:
%sql
SELECT NO_MUNICIPIO_ESC, count(*) as total
FROM classes.bronze.resultados_2024
WHERE NO_MUNICIPIO_ESC LIKE '%�%'  -- Mudou aqui: LIKE e aspas
GROUP BY NO_MUNICIPIO_ESC
ORDER BY total DESC